In [1]:
from environs import Env

from models import (
    db,
    Asset,
    DailyPrice,
)

env = Env()
env.read_env()

True

In [2]:
db.bind(provider='postgres',
        user=env.str('DB_USER'),
        password=env.str('DB_PASSWORD'),
        host=env.str('DB_HOST'),
        database=env.str('DB'),
       )

db.generate_mapping()

In [3]:
from pony import orm

query = orm.select(a for a in Asset)
query.first()

Asset['000020']

In [21]:
from datetime import date, timedelta
from dateutil.parser import parse

asset = '000120'
sm_date = date(2001, 8, 3)
del_days = timedelta(days=5)

query = orm.select(dp for dp in DailyPrice if dp.asset.code == asset) \
    .filter(lambda x: x.date > sm_date - del_days and x.date < sm_date + del_days)

for d in query:
    print(d.date, d.close)

2001-08-07 10350.0
2001-08-06 11050.0
2001-08-03 11950.0
2001-08-02 1730.0
2001-08-01 1730.0
2001-07-31 1730.0
2001-07-30 1730.0


In [20]:
from dateutil.parser import parse

parse('2001-07-20').date()

datetime.date(2001, 7, 20)

In [32]:
# update test


query = orm.select(dp for dp in DailyPrice 
                   if dp.asset.code == asset and dp.date < parse('2001-07-20').date())


for dp in query:
    dp.open *= 6
    dp.high *= 6
    dp.low *= 6
    dp.close *= 6
    dp.volume /= 6
        
orm.commit()

In [34]:
# update test


query = orm.select(dp for dp in DailyPrice if dp.asset.code == asset) \
    .filter(lambda x: x.date >= parse('2001-07-16') and x.date <= parse('2001-08-02'))

for dp in query:
    dp.delete()
    
orm.commit()

In [37]:
asset

'000120'

In [39]:
# is_adjusted

query = orm.select(a for a in Asset if a.code == asset)

for a in query:
    print(a.code, a.name)
    a.is_adjusted = True
    
orm.commit()

000120 CJ대한통운
